#**ACCELERATED IMAGE RECOMMENDATION SYSTEM**

### **PART 2: GPU ACCELERATING BASELINE MODEL** 

Import Statements

In [1]:
import os
import time
from math import *
import numpy as np
import cupy as cp

In [ ]:
!nvidia-smi

### Loading Features of Database Images and Query Image extracted from:
1. RGB3D Histogram and Hu Moments
2. MobileNet Features

In [ ]:
queryVector_rgb3d = np.load('/content/blue359_rgb.npy')
queryVector_mobilenet = np.load('/content/blue359_mobilenet.npy')

features = np.load('/content/drive/MyDrive/image_features.npy',allow_pickle=True) 
features_flat = features.flat[0]
features_rgb3d = list(features_flat.items())  

features = np.load('/content/mobilenet_features.npy',allow_pickle=True)   
features_flat = features.flat[0]
features_mobilenet = list(features_flat.items())

### Accelerated Distance Measures

### 1. Euclidean Distance

In [ ]:
def euclidean_distance(queryVector, feature):   
    try:
      img = feature[0]
      vec = feature[1]
      X_gpu = cp.asarray(vec)
      queryVector_gpu = cp.asarray(queryVector)
      dist = cp.sum((X_gpu[:,None,:] - queryVector_gpu[None,:,:])**2, axis=2)**0.5
      return img,cp.asnumpy(dist)
    except:
      pass
    

### Manhattan Distance

In [ ]:
def manhattan_distance(queryVector, feature): 
  try:
    img = feature[0]
    vec = feature[1]
    X_gpu = cp.asarray(vec)
    queryVector_gpu = cp.asarray(queryVector)
    temp =cp.sum((queryVector_gpu - X_gpu))
    dist = cp.abs(temp)
    return img,cp.asnumpy(dist)
  except:
      pass

### Chi Squared

In [ ]:
def chi_squared(queryVector, feature):
  try:
    img = feature[0]
    vec = feature[1]
    X_gpu = cp.asarray(vec)
    queryVector_gpu = cp.asarray(queryVector)

    num = ((X_gpu[:,None,:] - queryVector_gpu[None,:,:])**2)
    den = (X_gpu[:,None,:] + queryVector_gpu[None,:,:])
    dist = cp.sum(num/den)
       
    return img,cp.asnumpy(dist)

  except:
      pass

### Cosine Similarity

In [ ]:
def cosine(queryVector, feature): 
  try:
    img = feature[0]
    vec = feature[1]
    X_gpu = cp.asarray(vec)
    queryVector_gpu = cp.asarray(queryVector)


    dot_product = cp.dot(queryVector_gpu, cp.transpose(X_gpu))
    norm_1 = cp.linalg.norm(queryVector_gpu)  
    norm_2 = cp.linalg.norm(X_gpu)
    dist =  dot_product / (norm_1 * norm_2)
    dist = 1-dist
       
    return img,cp.asnumpy(dist)
  except:
      pass

### Accelerated Feature Comparison for Image features extracted using RGB3D histogram and Hu Moements

In [ ]:
start = time.perf_counter()
search_results = euclidean_distance(queryVector_rgb3d, features_rgb3d)
end = time.perf_counter()
print(f"Time for Accelerated query search - euclidean: {end-start}")



start = time.perf_counter()
search_results = manhattan_distance(queryVector_rgb3d, features_rgb3d)
end = time.perf_counter()
print(f"Time for Accelerated query search - manhattan: {end-start}")


start = time.perf_counter()
search_results = chi_squared(queryVector_rgb3d, features_rgb3d)
end = time.perf_counter()
print(f"Time for Accelerated query search - chi_squared: {end-start}")



start = time.perf_counter()
search_results = cosine(queryVector_rgb3d, features_rgb3d)
end = time.perf_counter()
print(f"Time for Accelerated query search - cosine: {end-start}")

### Accelerated Feature Comparison for Image features extracted using MobileNet

In [ ]:
start = time.perf_counter()
search_results = euclidean_distance(queryVector_mobilenet, features_mobilenet)
end = time.perf_counter()
print(f"Time for Accelerated query search - euclidean: {end-start}")



start = time.perf_counter()
search_results = manhattan_distance(queryVector_mobilenet, features_mobilenet)
end = time.perf_counter()
print(f"Time for Accelerated query search - manhattan: {end-start}")


start = time.perf_counter()
search_results = chi_squared(queryVector_mobilenet, features_mobilenet)
end = time.perf_counter()
print(f"Time for Accelerated query search - chi_squared: {end-start}")



start = time.perf_counter()
search_results = cosine(queryVector_mobilenet, features_mobilenet)
end = time.perf_counter()
print(f"Time for Accelerated query search - cosine: {end-start}")